# Project: Financial Product Complaint Classification and Summarization
## Step-by-Step Implementation using Generative AI (Mistral)


**⛳ Requirement -1** :

**Zero-Shot Prompting for Text Classification**

**The requirement :**


Setting Up for Prompt Engineering with Mistral Model

  - Install neccessary libraries

  - Importing Libaries and Setting up Mistral Model

  -  Define Prompt Template, System Message, **generate_prompt()**

  - Create mistral_response column using **generate_mistral_response()**

  - Create mistral_response_cleaned using **extract_category()**

  - Calculate F1 Score

  - Explain the F1 difference






In [1]:
!pip install openai==0.28 --upgrade


In [10]:
# STEP 1: Importing libraries
import pandas as pd
import openai
import re
from sklearn.metrics import f1_score

# Adding my OpenRouter key


openai.api_key = "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"
openai.api_base = "https://openrouter.ai/api/v1"


# STEP 2: Loading the dataset from sample_data/ashish (i uploaded the csv file first)
df = pd.read_csv("sample_data/ashish/Complaints_classification.csv")
df.head()

# STEP 3: Defining system prompt and helper functions
system_message = "You are a helpful assistant that classifies financial complaints into categories like credit card, credit reporting, retail banking, student loan, etc."

def generate_prompt(text):
    return f"Classify this financial complaint into one category:\n\n{text}\n\nCategory:"

def generate_mistral_response(narrative):
    try:
        response = openai.ChatCompletion.create(
            model="mistralai/mistral-7b-instruct",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": generate_prompt(narrative)}
            ]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return str(e)

# STEP 4: Testing on a smaller sample to avoid rate limits
df_sample = df.head(10).copy()
df_sample['mistral_response'] = df_sample['narrative'].apply(generate_mistral_response)

# STEP 5: Extracting clean required category (e.g., "credit card" only)
def extract_category(response):
    match = re.search(r"(credit card|credit reporting|retail banking|student loan|checking account|loan|mortgage|money transfer|auto loan)",
                      response, re.IGNORECASE)
    return match.group(1).lower() if match else "unknown"

df_sample['mistral_response_cleaned'] = df_sample['mistral_response'].apply(extract_category)

# STEP 6: Fixing label mismatch: converting underscores to spaces
y_true = df_sample['product'].str.lower().str.replace("_", " ")
y_pred_raw = df_sample['mistral_response'].str.lower()
y_pred_clean = df_sample['mistral_response_cleaned']

# STEP 7: Calculating F1 Scores
f1_raw = f1_score(y_true, y_pred_raw, average='macro', zero_division=0)
f1_clean = f1_score(y_true, y_pred_clean, average='macro', zero_division=0)

print("----------------Zero-Shot Prompting for Text Classification-----------------------")
print(f"F1 Score (Raw) is : {f1_raw:.4f}")
print(f"F1 Score (Cleaned) is : {f1_clean:.4f}")
print("---------------------------------------")

# additional : View predictions, mistral_response
# pd.set_option('display.max_colwidth', None) // using this line to show full content in columns.
pd.set_option('display.max_colwidth', None)

df_sample[['product', 'narrative', 'mistral_response', 'mistral_response_cleaned']]


----------------Zero-Shot Prompting for Text Classification-----------------------
F1 Score (Raw) is : 0.0000
F1 Score (Cleaned) is : 0.2667
---------------------------------------


,product,narrative,mistral_response,mistral_response_cleaned
0,credit_card,purchase order day shipping amount receive product week sent followup email exact verbiage paid two day shipping received order company responded im sorry inform due unusually high order volume order shipped several week stock since early due high demand although continuing take order guaranteeing receive order place due time mask order exact shipping date right however guarantee ship soon soon delivers product u getting small shipment shipping first come first served basis appreciate patience fulfill order quickly recommend keeping order lose place line cancel distributor stock moment prefer cancel please note ask via email cancel accordance cancellation policy agreed checkout electronic inventory online requested order canceled refund issued canceled order sent verification order canceled refunded item particulate respirator refunded subtotal shipping tax total usd visa ending refund called disputed amount stated nothing needed submitted address issue recharged item removing called back dispute amount transaction rebillmerchandiserobert ca purchased thu posted wed purchased appears statement transaction rebill ca u followed see status case submitted documentation showing canceled order supposed submit refund called back speak manager case stated dispute ruled favor charge removed card capital one removed purchase bill purchase adjustmentmerchandiserobert j posted fri purchased appears statement purchase adjustment capital one recharges amount transaction rebillmerchandiserobert j purchased thu posted mon purchased appears statement transaction rebill called capital one requested recharge stated visa ruled case pretended remove purchase knew anything case manager ruling favor,"This complaint appears to be related to a dispute over a purchase or a charging issue with a credit card, specifically involving Capitol One and merchandiser Robert J. This complaint involves a refund dispute, a recharge, and a transaction rebill, which are common issues that typically fall under the credit card category. However, it's important to note that the specific details about the purchase (retail banking, specific product, etc.) are not explicitly mentioned. Nonetheless, the main focus of the complaint seems to be a credit card-related issue.",credit card
1,credit_card,forwarded message date tue subject please investigate comenity bank retailer card scam sent hello name scammed comenity bank credit card provider company childrens place new york forever victoria secret original credit comenity bank lower limit began charge overage fee along late fee began pay close attention card find limit also changed well incurring overage late fee reached company comenity bank stated would change credit limit original limit reached told summit payment account corrected comenity bank credit card impacted credit score plummeted negative status im currently paying price due corruption affected detrimental way debt due company charging overage fee well late fee even initial credit limit fluctuating tremendously company charge major fee account willing correct account nervous said attorney reason im reaching im employee company ruining credit plz help name contact info thank,"Based on the details provided, this complaint appears to be related to a Retail Banking issue, specifically concerning a Credit Card. The specific issue seems to revolve around unauthorized charges, fluctuating credit limit, overage fees, and late fees from Comenity Bank for a Credit Card issued for The Children's Place, Victoria's Secret, or Forever 21 retailers. This complaint possibly also involves a negative impact on the credit score.",retail banking
2,retail_banking,forwarded message cc sent friday pdt subject final legal payment well fargo well fargo clearly wrong need look actually opened account see court hearing several different government agency reporting well including branch put open account numerous fra

 ========================================================

**Observations & Results for Zero-Shot Prompting**

I implemented zero-shot classification using the Mistral-7B-Instruct model via OpenRouter API. A system message and prompt template were used to generate responses for financial complaint narratives. Also the LLM-generated output was post-processed using a category extraction function.

F1 Score (Raw): 0.0000

F1 Score (Cleaned): 0.2667

**My Observation:**

The raw Mistral responses often contained full-sentence explanations, making them unsuitable for direct label comparison. After applying regular expressions to extract standardized product categories from the responses, the cleaned predictions yielded a significantly higher F1 score. This highlights the importance of post-processing when using generative models for classification tasks.




================================================================================

---------------------------------------------------------------------------


**⛳ ⛳  Requirement #2** :

**Few-Shot Prompting for Text Classification**

- Define system_message, first_turn_template, example_template, prediction_template

- Create few-shot prompt using gold examples

- Create mistral_response and mistral_response_cleaned

- Calculate F1 Score

- Share observation comparing few-shot vs zero-shot






In [11]:
import pandas as pd
from sklearn.metrics import f1_score

# System message
system_message_few_shot = (
    "You are a helpful assistant that classifies financial complaints into product categories like:\n"
    "- credit card\n- credit reporting\n- retail banking\n- mortgage\n- loan\n"
    "Return only the category name. Respond concisely."
)

# Few-shot examples
few_shot_examples = [
    {"narrative": "Unauthorized credit card transaction and no resolution from bank", "label": "credit card"},
    {"narrative": "Incorrect information in credit report and Experian not fixing it", "label": "credit reporting"},
    {"narrative": "Bank charged overdraft fees despite having sufficient balance", "label": "retail banking"},
]

# Prompt builder
def build_few_shot_prompt(narrative):
    prompt = ""
    for ex in few_shot_examples:
        prompt += f"Complaint: {ex['narrative']}\nCategory: {ex['label']}\n\n"
    prompt += f"Complaint: {narrative}\nCategory:"
    return prompt

# API call
def generate_few_shot_response(narrative):
    try:
        response = openai.ChatCompletion.create(
            model="mistralai/mistral-7b-instruct",
            messages=[
                {"role": "system", "content": system_message_few_shot},
                {"role": "user", "content": build_few_shot_prompt(narrative)}
            ]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return str(e)

# Run on a small sample
df_sample = df.head(10).copy()

# Apply model
df_sample['mistral_response'] = df_sample['narrative'].apply(generate_few_shot_response)
df_sample['mistral_response_cleaned'] = df_sample['mistral_response'].apply(extract_category)

# Prepare and evaluate
y_true = df_sample['product'].str.lower().str.replace("_", " ")
y_pred_clean = df_sample['mistral_response_cleaned'].str.lower()

f1_few_shot = f1_score(y_true, y_pred_clean, average='macro', zero_division=0)
print(f"F1 Score (Few-Shot Cleaned): {f1_few_shot:.4f}")

# ✅ Display table with relevant columns for presentation
df_sample_display = df_sample[['narrative', 'mistral_response', 'mistral_response_cleaned']]
df_sample_display.head(3)  # 👈 show only first 3 for slide/screenshot


F1 Score (Few-Shot Cleaned): 0.4308


,narrative,mistral_response,mistral_response_cleaned
0,purchase order day shipping amount receive product week sent followup email exact verbiage paid two day shipping received order company responded im sorry inform due unusually high order volume order shipped several week stock since early due high demand although continuing take order guaranteeing receive order place due time mask order exact shipping date right however guarantee ship soon soon delivers product u getting small shipment shipping first come first served basis appreciate patience fulfill order quickly recommend keeping order lose place line cancel distributor stock moment prefer cancel please note ask via email cancel accordance cancellation policy agreed checkout electronic inventory online requested order canceled refund issued canceled order sent verification order canceled refunded item particulate respirator refunded subtotal shipping tax total usd visa ending refund called disputed amount stated nothing needed submitted address issue recharged item removing called back dispute amount transaction rebillmerchandiserobert ca purchased thu posted wed purchased appears statement transaction rebill ca u followed see status case submitted documentation showing canceled order supposed submit refund called back speak manager case stated dispute ruled favor charge removed card capital one removed purchase bill purchase adjustmentmerchandiserobert j posted fri purchased appears statement purchase adjustment capital one recharges amount transaction rebillmerchandiserobert j purchased thu posted mon purchased appears statement transaction rebill called capital one requested recharge stated visa ruled case pretended remove purchase knew anything case manager ruling favor,credit card (This complaint seems to be about an unauthorized credit card transaction and a dispute about it with Capital One.),credit card
1,forwarded message date tue subject please investigate comenity bank retailer card scam sent hello name scammed comenity bank credit card provider company childrens place new york forever victoria secret original credit comenity bank lower limit began charge overage fee along late fee began pay close attention card find limit also changed well incurring overage late fee reached company comenity bank stated would change credit limit original limit reached told summit payment account corrected comenity bank credit card impacted credit score plummeted negative status im currently paying price due corruption affected detrimental way debt due company charging overage fee well late fee even initial credit limit fluctuating tremendously company charge major fee account willing correct account nervous said attorney reason im reaching im employee company ruining credit plz help name contact info thank,"credit card, retail banking, credit reporting (multiple issues)",credit card
2,forwarded message cc sent friday pdt subject final legal payment well fargo well fargo clearly wrong need look actually opened account see court hearing several different government agency reporting well including branch put open account numerous fraudulent charge debit card connected account clearly due fact well fargo maintain clearly required encryption card keep secure adding damage request small claim lawsuit time take away running business branch required come branch multiple time get fraudulent charge removed actually present strong case proper consumer protection agency well fargo immediately barred issuing account debit card insure agency card adequate prevent fraudulent use also requesting punitive treble damage sum hour take work handle lawsuit along legal cost filing lawsuit county small claim court today well fargo outrageous fraudulent conduct clearly warrant punitive damage also sharing every possible consumer protection agency find would think well fargo enough adverse publicity fraudulent conduct last year introduce small claim suit clearly establishing well fargo established bu

#### **Results of Few-Shot Prompting for Text Classification:**
- F1 Score (Few-Shot Cleaned): 0.4308

#### **Observation**
The examples helped the model understand categories better. More carefully chosen examples can improve this further.


---------------------------------------------------------------------------


**⛳ ⛳ ⛳ Requirement #3**

**Zero-Shot Prompting for Text Summarization**

- Define **system_message**
- Generate **mistral_response** as summaries
- Evaluate with BERTScore
- Write final observation


In [ ]:
!pip install bert_score


In [13]:
system_message_summarize = (
    "You are a helpful assistant that summarizes financial product complaints in 2–3 lines. "
    "Make the summary concise, factual, and easy to understand for customer service teams."
)

def generate_summary_prompt(text):
    return f"Summarize the following complaint:\n\n{text}\n\nSummary:"

def generate_summary(narrative):
    try:
        response = openai.ChatCompletion.create(
            model="mistralai/mistral-7b-instruct",
            messages=[
                {"role": "system", "content": system_message_summarize},
                {"role": "user", "content": generate_summary_prompt(narrative)}
            ]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return str(e)

# Take a sample of 10 complaints
df_summary = df.head(10).copy()
df_summary['mistral_response'] = df_summary['narrative'].apply(generate_summary)

# ------------------------
# ✅ FIX: Ensure clean input before BERTScore
# ------------------------

from bert_score import score

# Drop rows where narrative or mistral_response is missing
df_summary = df_summary.dropna(subset=['narrative', 'mistral_response'])

# Ensure both columns are strings
df_summary['narrative'] = df_summary['narrative'].astype(str)
df_summary['mistral_response'] = df_summary['mistral_response'].astype(str)

# Evaluate BERTScore
P, R, F1 = score(
    df_summary['mistral_response'].tolist(),
    df_summary['narrative'].tolist(),
    lang="en",
    verbose=True
)
print("---------------------------------------------")
print(f" 🔍 Average BERTScore F1: {F1.mean():.4f}")
print("---------------------------------------------")

# Show side-by-side comparison of gold summary vs model summary
df_summary[['narrative', 'summary', 'mistral_response']].head(3)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 51.54 seconds, 0.19 sentences/sec
---------------------------------------------
 🔍 Average BERTScore F1: 0.8168
---------------------------------------------


,narrative,summary,mistral_response
0,purchase order day shipping amount receive product week sent followup email exact verbiage paid two day shipping received order company responded im sorry inform due unusually high order volume order shipped several week stock since early due high demand although continuing take order guaranteeing receive order place due time mask order exact shipping date right however guarantee ship soon soon delivers product u getting small shipment shipping first come first served basis appreciate patience fulfill order quickly recommend keeping order lose place line cancel distributor stock moment prefer cancel please note ask via email cancel accordance cancellation policy agreed checkout electronic inventory online requested order canceled refund issued canceled order sent verification order canceled refunded item particulate respirator refunded subtotal shipping tax total usd visa ending refund called disputed amount stated nothing needed submitted address issue recharged item removing called back dispute amount transaction rebillmerchandiserobert ca purchased thu posted wed purchased appears statement transaction rebill ca u followed see status case submitted documentation showing canceled order supposed submit refund called back speak manager case stated dispute ruled favor charge removed card capital one removed purchase bill purchase adjustmentmerchandiserobert j posted fri purchased appears statement purchase adjustment capital one recharges amount transaction rebillmerchandiserobert j purchased thu posted mon purchased appears statement transaction rebill called capital one requested recharge stated visa ruled case pretended remove purchase knew anything case manager ruling favor,"The customer made a purchase order with an agreement for two-day shipping but didn't receive the product due to high demand and delayed shipment. The company informed the customer about the delay, suggesting to keep the order or cancel it. The customer wanted to cancel it and followed the company's policy by requesting the cancellation via email. After the order was canceled, the company verified the cancellation and issued a refund. However, the customer noted a discrepancy in refund claiming they were () charged for the canceled product. Though they disputed the issue, it was initially ruled in the company's favor. Capital One removed the mention of the purchase from the bill, but it reappeared later. The customer is now in a dispute over the recharged amount and is waiting for the final ruling on the case.","The customer placed an order on Thursday, paid for two-day shipping but received the order a week later due to unusually high order volume and high demand. The customer received a follow-up email, but was not satisfied with the delay and wanted to cancel the order. The company responded apologetically, stating that they would ship soon, but the customer received a small shipment first on a first-come-first-served basis. The customer expressed patience but requested a faster delivery and recommended keeping the order in the queue rather than canceling it.\n\nThe customer later disputed the charge with their bank, claiming the purchase was not acknowledged or refunded, and provided documentation showing the canceled order. The bank, however, recharged the amount and stated the transaction was a rebill from merchant Robert CA. The customer was unhappy and asked to speak with a manager, who ruled in favor of the charge removal. The customer received a purchase adjustment from Capital One."
1,forwarded message date tue subject please investigate comenity bank retailer card scam sent hello name scammed comenity bank credit card provider company childrens place new york forever victoria secret original credit comenity bank lower limit began charge overage fee along late fee began pay close attention card find limit also changed well incurring overage late fee reached company comenity bank stated would change credit 

The Average BERTScore F1: 0.8145

The score is good, which means Mistral’s summaries match the original complaints well. Even though the summaries are short or to the point, they still keep the main idea clear and preserve important information.
